<center>
<h1>基于pytorch + LSTM 的古诗生成</h1>
</center>

### 作业介绍: 
本课程使用pytorch框架, 完成NLP任务:古诗生成,使用的模型为 LSTM, 并训练了词向量, 支持随机古诗和藏头诗生成, 并且生成的古诗具有多变性。

<br>

### 导包:

In [1]:
import os
import numpy as np
import pickle
import torch
import torch.nn as nn
from gensim.models.word2vec import Word2Vec
from torch.utils.data import Dataset, DataLoader

<br>

### 生成切分文件:

In [2]:
def split_text(file="poetry_7.txt", train_num=6000):
    all_data = open(file, "r", encoding="utf-8").read()
    with open("split_7.txt", "w", encoding="utf-8") as f:
        split_data_7 = " ".join(all_data)
        f.write(split_data_7)
    return split_data_7[:train_num * 64]

<br>

### 训练词向量:

In [3]:
def train_vec(split_file="split_7.txt", org_file="poetry_7.txt", train_num=6000):
    param_file = "word_vec.pkl"
    org_data = open(org_file, "r", encoding="utf-8").read().split("\n")[:train_num]
    if os.path.exists(split_file):
        all_data_split = open(split_file, "r", encoding="utf-8").read().split("\n")[:train_num]
    else:
        all_data_split = split_text().split("\n")[:train_num]

    if os.path.exists(param_file):
        return org_data, pickle.load(open(param_file, "rb"))

    models = Word2Vec(all_data_split, vector_size=128, workers=7, min_count=1)
    pickle.dump([models.syn1neg, models.wv.key_to_index, models.wv.index_to_key], open(param_file, "wb"))
    return org_data, (models.syn1neg, models.wv.key_to_index, models.wv.index_to_key)

<br>

### 构建数据集:

In [4]:
class Poetry_Dataset(Dataset):
    def __init__(self, w1, word_2_index, all_data):
        self.w1 = w1
        self.word_2_index = word_2_index
        self.all_data = all_data

    def __getitem__(self, index):
        a_poetry = self.all_data[index]

        a_poetry_index = [self.word_2_index[i] for i in a_poetry]
        xs = a_poetry_index[:-1]
        ys = a_poetry_index[1:]
        xs_embedding = self.w1[xs]

        return xs_embedding, np.array(ys).astype(np.int64)

    def __len__(self):
        return len(self.all_data)

<br>

### 模型构建:

In [5]:
class Poetry_Model_lstm(nn.Module):
    def __init__(self, hidden_num, word_size, embedding_num):
        super().__init__()

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        ######定义模型######
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.hidden_num = hidden_num

        self.lstm = nn.LSTM(input_size=embedding_num, hidden_size=hidden_num, batch_first=True, num_layers=2,
                            bidirectional=False)
        self.dropout = nn.Dropout(0.3)
        self.flatten = nn.Flatten(0, 1)
        self.linear = nn.Linear(hidden_num, word_size)
        self.cross_entropy = nn.CrossEntropyLoss()


    def forward(self, xs_embedding, h_0=None, c_0=None):
        if h_0 == None or c_0 == None:
            h_0 = torch.tensor(np.zeros((2, xs_embedding.shape[0], self.hidden_num), dtype=np.float32))
            c_0 = torch.tensor(np.zeros((2, xs_embedding.shape[0], self.hidden_num), dtype=np.float32))
        h_0 = h_0.to(self.device)
        c_0 = c_0.to(self.device)
        xs_embedding = xs_embedding.to(self.device)
        ######定义模型######
        hidden, (h_0, c_0) = self.lstm(xs_embedding, (h_0, c_0))
        hidden_drop = self.dropout(hidden)
        hidden_flatten = self.flatten(hidden_drop)
        pre = self.linear(hidden_flatten)

        return pre, (h_0, c_0)

<br>

### 自动生成古诗:


In [9]:
def generate_poetry_auto():
    result = ""
    word_index = np.random.randint(0, word_size, 1)[0]

    result += index_2_word[word_index]
    h_0 = torch.tensor(np.zeros((2, 1, hidden_num), dtype=np.float32))
    c_0 = torch.tensor(np.zeros((2, 1, hidden_num), dtype=np.float32))

    for i in range(31):
        word_embedding = torch.tensor(w1[word_index][None][None])
        pre, (h_0, c_0) = model(word_embedding, h_0, c_0)
        word_index = int(torch.argmax(pre))
        result += index_2_word[word_index]

    return result


<br>

### 藏头诗生成:

In [10]:
def generate_poetry_acrostic():
    input_text = input("请输入四个汉字：")[:4]
    result = ""
    punctuation_list = ["，", "。", "，", "。"]
    for i in range(4):
        result += input_text[i]
        h_0 = torch.tensor(np.zeros((2, 1, hidden_num), dtype=np.float32))
        c_0 = torch.tensor(np.zeros((2, 1, hidden_num), dtype=np.float32))
        word = input_text[i]
        for j in range(6):
            word_index = word_2_index[word]
            word_embedding = torch.tensor(w1[word_index][None][None])
            pre , (h_0,c_0) = model(word_embedding,h_0,c_0)
            word = word_2_index[int(torch.argmax(pre))]
            result += word

    return result

<br>

### 主函数: 定义参数, 模型, 优化器, 模型训练

In [11]:
if __name__ == "__main__":

    all_data, (w1, word_2_index, index_2_word) = train_vec(train_num=300)

    batch_size = 32
    epochs = 100
    lr = 0.01
    hidden_num = 128
    word_size, embedding_num = w1.shape

    dataset = Poetry_Dataset(w1, word_2_index, all_data)
    dataloader = DataLoader(dataset, batch_size)

    model = Poetry_Model_lstm(hidden_num, word_size, embedding_num)
    model = model.to(model.device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    for e in range(epochs):
        for batch_index, (batch_x_embedding, batch_y_index) in enumerate(dataloader):
            model.train()
            batch_x_embedding = batch_x_embedding.to(model.device)
            batch_y_index = batch_y_index.to(model.device)

            #模型预测
            pre, _ = model(batch_x_embedding)
            #计算损失
            loss = model.cross_entropy(pre, batch_y_index.reshape(-1))
            # 梯度反传 , 梯度累加, 但梯度并不更新, 梯度是由优化器更新的
            loss.backward()
            # 使用优化器更新梯度
            optimizer.step()
            # 梯度清零
            optimizer.zero_grad()
            
            if batch_index % 100 == 0:
                # model.eval()
                print(f"loss:{loss:.3f}")
                print(generate_poetry_auto())

loss:7.707
舠，，，，，，，，。，，，，，，，，，，，，。，，，，。，，。，
loss:6.962
则，。。，。。，，。。。。，，。。，。，。。。，。。，。，，，。
loss:6.800
立，。，，。。。，。，，，。，。。。。。。。。。。。，，。。，。
loss:6.725
窅。。，有，，有。有，。。，色，色，有，有。，，，有，色，，有，
loss:6.644
马山花花花，山山山，花海，山山山山山花花，山花，花山山山山花花，
loss:6.583
壶山山山山，山，山山山，山山，山海，山山，山，山山海，山山海，山
loss:6.414
程山风风山无，，风，，，，，，，，，，，，，，，，，，，，，，，
loss:6.292
甲有三海不，，山，，，，，，，，，天。。山。一。。，一。，，，，
loss:6.163
半色三三一天，，来一一。一一。。。一一山。。。一一。。。一一。。
loss:6.110
双帆三路六水，，一山山一天。一一。山一无天。，一山一无无。。山一
loss:6.051
缘门三光斗，海天，一一，一天。一天。一一。一一。一一。一一。一一
loss:5.956
开得三十斗斗，一山不一一。一山一来。一一。一一水，一风一人。一来
loss:5.864
嗤台下车自斗险，一山一山人色。一山一一一。一山一人水。一来一似人
loss:5.792
陈榜风车斗天，天山不一一水。一来风来不人，一风一子一水。海山一人
loss:5.737
慵门由气千海菲，一花一山一天。一来一花一天。一山一年一水，一来一
loss:5.676
人人风风三远，海山一山一。金风一有无，一山一山不来。一来一有一无
loss:5.601
优今极树海有，萧山一山一青。一山风山一无。海山一里天时。一来一山
loss:5.574
弼门十草政苇阳，一山衔子一浮，。山烟垒一人。一来一声不节。一来一
loss:5.532
慧门十气斗注开，匹山无榔一水花。一山一山一人，一山一人频心。一来
loss:5.509
幔得下气接峰菲，砌霄一咏一山中。海时一望不来，海山烟山无春。一山
loss:5.474
槟阙喧光海告，一山似光一天。一天一来一来来，一来一风一无来。一山
loss:5.408
静台风光自生，匹下一光一天。一山风时一水，一里应花不公。一是一声
loss:5.378
晨蛇红山六重垧，瘴下一阁旧水中。海来一影一

## 描述一下你的模型：


### 模型概述

该模型是一个基于深度学习的古诗生成系统，它利用word2vec技术将汉字表示为词向量，并通过LSTM（长短期记忆网络）和一个全连接层来构建预测模型。通过训练，该模型能够根据给定的前一个字预测下一个字，从而生成古诗。

### 模型细节

1. **汉字表示（word2vec）**

   - **输入**：汉字集合（例如，从大量古诗中提取的所有不重复汉字）。
   - **处理**：使用word2vec算法将每个汉字转换为一个固定维度的词向量。这些词向量能够捕捉到汉字之间的语义关系，例如相似的汉字在向量空间中会彼此靠近。
   - **输出**：一个汉字到词向量的映射表，以及每个汉字对应的词向量。

2. **封装数据**

   - **目的**：为了方便模型处理，需要将汉字及其对应的词向量封装成数据格式。
   - **处理**：为每个汉字的词向量分配一个唯一的索引，并创建一个字典来存储这些索引和词向量的对应关系。同时，将古诗文本转换为一系列索引，以便模型能够读取和预测。
   - **输出**：索引到词向量的映射字典，以及古诗文本转换为索引序列的数据集。

3. **构建模型**

   - **架构**：模型由一个LSTM层和一个全连接层组成。LSTM层负责捕捉序列数据中的长期依赖关系，而全连接层则用于将LSTM的输出转换为预测下一个字的概率分布。
   - **输入**：前一个字的词向量（通过索引从字典中检索得到）。
   - **输出**：下一个字的可能性的概率分布（通常是一个softmax层输出的概率向量）。

4. **训练**

   - **过程**：使用大量古诗文本作为训练数据，通过前向传播和反向传播算法来训练模型。在训练过程中，模型会尝试根据前一个字的词向量来预测下一个字的索引。
   - **损失函数**：通常使用交叉熵损失函数来衡量模型预测的准确性，并通过梯度下降等优化算法来更新模型的权重。
   - **终止条件**：训练过程会在达到预定的迭代次数、损失函数收敛到某个阈值或模型在验证集上的性能不再提升时终止。

5. **生成古诗**

   - **过程**：在训练完成后，模型可以根据给定的起始字或句子生成古诗。这通常是通过在模型中输入起始字的词向量，并反复使用模型的预测输出来生成下一个字来实现的。
   - **控制**：可以通过设置生成古诗的长度、使用特定的起始字或句子以及调整生成过程中的随机性来控制生成的古诗的风格和内容。

### 总结

该模型利用word2vec将汉字表示为词向量，并通过LSTM和全连接层构建了一个能够预测下一个字的深度学习模型。通过训练，该模型能够生成符合古诗风格的文本。这种模型在文学创作、自然语言处理等领域具有广泛的应用前景。